In [122]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import urlencode

In [123]:
toronto_DF = pd.read_csv('postalcode.csv')
display(toronto_DF)

,Borough,Postalcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...,...
98,York,M6C,Humewood-Cedarvale,43.693781,-79.428191
99,York,M6E,Caledonia-Fairbanks,43.689026,-79.453512
100,York,M6M,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
101,York,M6N,"Runnymede, The Junction North",43.673185,-79.487262


In [124]:
postal_codes = toronto_DF['Postalcode'].tolist()

In [125]:
len(toronto_DF)

103

In [126]:
postal_codes

['M4N',
 'M4P',
 'M4R',
 'M4S',
 'M4T',
 'M4V',
 'M5N',
 'M5P',
 'M5R',
 'M4W',
 'M4X',
 'M4Y',
 'M5A',
 'M5B',
 'M5C',
 'M5E',
 'M5G',
 'M5H',
 'M5J',
 'M5K',
 'M5L',
 'M5S',
 'M5T',
 'M5V',
 'M5X',
 'M6G',
 'M5W',
 'M4E',
 'M4K',
 'M4L',
 'M4M',
 'M7Y',
 'M4B',
 'M4C',
 'M4G',
 'M4H',
 'M4J',
 'M8V',
 'M8W',
 'M8X',
 'M8Y',
 'M8Z',
 'M9A',
 'M9B',
 'M9C',
 'M9P',
 'M9R',
 'M9V',
 'M9W',
 'M7R',
 'M2H',
 'M2J',
 'M2K',
 'M2L',
 'M2M',
 'M2N',
 'M2P',
 'M2R',
 'M3A',
 'M3B',
 'M3C',
 'M3H',
 'M3J',
 'M3K',
 'M3L',
 'M3M',
 'M3N',
 'M4A',
 'M5M',
 'M6A',
 'M6B',
 'M6L',
 'M9L',
 'M9M',
 'M7A',
 'M1B',
 'M1C',
 'M1E',
 'M1G',
 'M1H',
 'M1J',
 'M1K',
 'M1L',
 'M1M',
 'M1N',
 'M1P',
 'M1R',
 'M1S',
 'M1T',
 'M1V',
 'M1W',
 'M1X',
 'M6H',
 'M6J',
 'M6K',
 'M6P',
 'M6R',
 'M6S',
 'M6C',
 'M6E',
 'M6M',
 'M6N',
 'M9N']

In [127]:
# Assign the URL to a variable

url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=M1C&DGUIDlist=2021A0011M1C&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0"


# use the urlopen function to open the webpage
html = urllib.request.urlopen(url)

# show object html
html

In [128]:
# Create the BeautifulSoup object

html_to_parse = BeautifulSoup(html, "html.parser")

In [129]:
# create a list of tables.  There is only 1 table in this webpage

tables = html_to_parse.find_all("table")
print(f"Number of tables found: {len(tables)}")

Number of tables found: 1


In [130]:
df = pd.DataFrame(columns = ['postal_code', 'data', 'value'])

In [131]:
import urllib.request
from urllib.parse import urlparse, urlencode, parse_qsl, urlunparse
from bs4 import BeautifulSoup
import pandas as pd

# Example dataframe to store the results
population_by_postal = pd.DataFrame(columns=["Postalcode", "Title", "Population"])


# Loop through the postal codes
for i in postal_codes:
    url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E"
    params = {
        'SearchText': i,
        'DGUIDlist': '2021A0011'+i
    }

    # this part switches up the postal code parameter in the url
    url_parts = list(urlparse(url))
    query = dict(parse_qsl(url_parts[4]))
    query.update(params)
    
    url_parts[4] = urlencode(query)
    query_url = urlunparse(url_parts)
    
    try:
        # Open the URL and parse the HTML content
        html = urllib.request.urlopen(query_url)
        html_to_parse = BeautifulSoup(html, "html.parser")
        
        # Find all tables on the page
        tables = html_to_parse.find_all("table")
        
        # Check if any tables were found
        if len(tables) > 0:
            print(f"Number of tables found: {len(tables)}")
            title = f"2021A0011{i} - Population, 2021 - Counts - Total"
            
            # Find the relevant <td> with the title
            td = tables[0].find(attrs={"title": title})
            
            if td:
                population = float(td.text.replace(",", ""))  # Convert population to a number
                print(f"Postalcode: {i}, Population: {population}")
                population_by_postal.loc[len(population_by_postal.index)] = [i, title, population]
            else:
                print(f"Title not found for postal code: {i}")
        else:
            print(f"No tables found for postal code: {i}")
    
    except Exception as e:
        print(f"Error processing postal code {i}: {e}")

# Display the dataframe
print(population_by_postal)

Number of tables found: 1
Postalcode: M4N, Population: 16058.0
Number of tables found: 1
Postalcode: M4P, Population: 25057.0
Number of tables found: 1
Postalcode: M4R, Population: 11909.0
Number of tables found: 1
Postalcode: M4S, Population: 30754.0
Number of tables found: 1
Postalcode: M4T, Population: 10332.0
Number of tables found: 1
Postalcode: M4V, Population: 19273.0
Number of tables found: 1
Postalcode: M5N, Population: 16154.0
Number of tables found: 1
Postalcode: M5P, Population: 19791.0
Number of tables found: 1
Postalcode: M5R, Population: 26197.0
Number of tables found: 1
Postalcode: M4W, Population: 15296.0
Number of tables found: 1
Postalcode: M4X, Population: 19896.0
Number of tables found: 1
Postalcode: M4Y, Population: 36319.0
Number of tables found: 1
Postalcode: M5A, Population: 48978.0
Number of tables found: 1
Postalcode: M5B, Population: 17422.0
Number of tables found: 1
Postalcode: M5C, Population: 3149.0
Number of tables found: 1
Postalcode: M5E, Population: 1

In [132]:
display(population_by_postal)

,Postalcode,Title,Population
0,M4N,"2021A0011M4N - Population, 2021 - Counts - Total",16058.0
1,M4P,"2021A0011M4P - Population, 2021 - Counts - Total",25057.0
2,M4R,"2021A0011M4R - Population, 2021 - Counts - Total",11909.0
3,M4S,"2021A0011M4S - Population, 2021 - Counts - Total",30754.0
4,M4T,"2021A0011M4T - Population, 2021 - Counts - Total",10332.0
...,...,...,...
92,M6C,"2021A0011M6C - Population, 2021 - Counts - Total",23997.0
93,M6E,"2021A0011M6E - Population, 2021 - Counts - Total",37684.0
94,M6M,"2021A0011M6M - Population, 2021 - Counts - Total",42148.0
95,M6N,"2021A0011M6N - Population, 2021 - Counts - Total",40252.0


In [133]:
population_by_postal.to_csv('population_by_postal.csv', index=False)

In [134]:
population_by_postal_sorted = population_by_postal.sort_values(by="Population", ascending=False)
display(population_by_postal_sorted)

,Postalcode,Title,Population
49,M2N,"2021A0011M2N - Population, 2021 - Counts - Total",75100.0
69,M1B,"2021A0011M1B - Population, 2021 - Counts - Total",65555.0
45,M2J,"2021A0011M2J - Population, 2021 - Counts - Total",61761.0
21,M5V,"2021A0011M5V - Population, 2021 - Counts - Total",59912.0
42,M9V,"2021A0011M9V - Population, 2021 - Counts - Total",53878.0
...,...,...,...
57,M3K,"2021A0011M3K - Population, 2021 - Counts - Total",7865.0
50,M2P,"2021A0011M2P - Population, 2021 - Counts - Total",7546.0
17,M5H,"2021A0011M5H - Population, 2021 - Counts - Total",3248.0
14,M5C,"2021A0011M5C - Population, 2021 - Counts - Total",3149.0


In [135]:

#  dataframe to store the results
avg_income_by_postal = pd.DataFrame(columns=["Postalcode", "Title", "Avg Income"])


# Loop through the postal codes
for i in postal_codes:
    url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E"
    params = {
        'SearchText': i,
        'DGUIDlist': '2021A0011'+i
    }

    # this part switches up the postal code parameter in the url
    url_parts = list(urlparse(url))
    query = dict(parse_qsl(url_parts[4]))
    query.update(params)
    
    url_parts[4] = urlencode(query)
    query_url = urlunparse(url_parts)
    
    try:
        # Open the URL and parse the HTML content
        html = urllib.request.urlopen(query_url)
        html_to_parse = BeautifulSoup(html, "html.parser")
        
        # Find all tables on the page
        tables = html_to_parse.find_all("table")
        
        # Check if any tables were found
        if len(tables) > 0:
            print(f"Number of tables found: {len(tables)}")
            title = f"2021A0011{i} - Average after-tax income in 2020 among recipients ($) - Counts - Total"
            
            # Find the relevant <td> with the title
            td = tables[0].find(attrs={"title": title})
            
            if td:
                avg_after_tax_income = float(td.text.replace(",", ""))  # Convert population to a number
                print(f"Postalcode: {i}, Average after tax income: {avg_after_tax_income}")
                avg_income_by_postal.loc[len(avg_income_by_postal.index)] = [i, title, avg_after_tax_income]
            else:
                print(f"Title not found for postal code: {i}")
        else:
            print(f"No tables found for postal code: {i}")
    
    except Exception as e:
        print(f"Error processing postal code {i}: {e}")

# Display the dataframe
print(avg_income_by_postal)

Number of tables found: 1
Postalcode: M4N, Average after tax income: 129000.0
Number of tables found: 1
Postalcode: M4P, Average after tax income: 54950.0
Number of tables found: 1
Postalcode: M4R, Average after tax income: 85200.0
Number of tables found: 1
Postalcode: M4S, Average after tax income: 61050.0
Number of tables found: 1
Postalcode: M4T, Average after tax income: 105100.0
Number of tables found: 1
Postalcode: M4V, Average after tax income: 106200.0
Number of tables found: 1
Postalcode: M5N, Average after tax income: 81200.0
Number of tables found: 1
Postalcode: M5P, Average after tax income: 84700.0
Number of tables found: 1
Postalcode: M5R, Average after tax income: 85900.0
Number of tables found: 1
Postalcode: M4W, Average after tax income: 115500.0
Number of tables found: 1
Postalcode: M4X, Average after tax income: 41480.0
Number of tables found: 1
Postalcode: M4Y, Average after tax income: 47440.0
Number of tables found: 1
Postalcode: M5A, Average after tax income: 515

In [136]:
display(avg_income_by_postal)

,Postalcode,Title,Avg Income
0,M4N,2021A0011M4N - Average after-tax income in 202...,129000.0
1,M4P,2021A0011M4P - Average after-tax income in 202...,54950.0
2,M4R,2021A0011M4R - Average after-tax income in 202...,85200.0
3,M4S,2021A0011M4S - Average after-tax income in 202...,61050.0
4,M4T,2021A0011M4T - Average after-tax income in 202...,105100.0
...,...,...,...
91,M6C,2021A0011M6C - Average after-tax income in 202...,59800.0
92,M6E,2021A0011M6E - Average after-tax income in 202...,40800.0
93,M6M,2021A0011M6M - Average after-tax income in 202...,36680.0
94,M6N,2021A0011M6N - Average after-tax income in 202...,38760.0


In [137]:
avg_income_by_postal.to_csv('avg_income_by_postal.csv', index=False)

In [138]:
avg_income_by_postal_sorted = avg_income_by_postal.sort_values(by="Avg Income", ascending=False)

display(avg_income_by_postal_sorted)

,Postalcode,Title,Avg Income
0,M4N,2021A0011M4N - Average after-tax income in 202...,129000.0
9,M4W,2021A0011M4W - Average after-tax income in 202...,115500.0
5,M4V,2021A0011M4V - Average after-tax income in 202...,106200.0
4,M4T,2021A0011M4T - Average after-tax income in 202...,105100.0
34,M8X,2021A0011M8X - Average after-tax income in 202...,93900.0
...,...,...,...
71,M1G,2021A0011M1G - Average after-tax income in 202...,33840.0
42,M9V,2021A0011M9V - Average after-tax income in 202...,33040.0
82,M1V,2021A0011M1V - Average after-tax income in 202...,32920.0
60,M3N,2021A0011M3N - Average after-tax income in 202...,31240.0


In [139]:

avg_income_by_postal = avg_income_by_postal.drop(columns=["Title"], errors="ignore")
population_by_postal = population_by_postal.drop(columns=["Title"], errors="ignore")



In [140]:
combined_df = pd.merge(avg_income_by_postal, population_by_postal, on="Postalcode")

# Now merge the combined DataFrame with toronto_DF on "Postalcode"
final_combined_df = pd.merge(combined_df, toronto_DF, on="Postalcode", how="left")  # Use how="left" to keep all rows from combined_df

# Sort the final combined DataFrame by both "Avg Income" and "Population" in descending order
final_combined_sorted = final_combined_df.sort_values(by=["Avg Income", "Population"], ascending=False)

# Display the top row which has the highest values for both
print(final_combined_sorted.head(1))  # This will show the postal code with the highest income and population


  Postalcode  Avg Income  Population          Borough  Neighbourhood  \
0        M4N    129000.0     16058.0  Central Toronto  Lawrence Park   

   Latitude  Longitude  
0  43.72802  -79.38879  


In [141]:
final_combined_sorted.head(20)

,Postalcode,Avg Income,Population,Borough,Neighbourhood,Latitude,Longitude
0,M4N,129000.0,16058.0,Central Toronto,Lawrence Park,43.728020,-79.388790
9,M4W,115500.0,15296.0,Downtown Toronto,Rosedale,43.679563,-79.377529
5,M4V,106200.0,19273.0,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
4,M4T,105100.0,10332.0,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
34,M8X,93900.0,10624.0,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
29,M4G,87500.0,19598.0,East York,Leaside,43.709060,-79.363452
8,M5R,85900.0,26197.0,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
62,M5M,85700.0,25495.0,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
2,M4R,85200.0,11909.0,Central Toronto,North Toronto West,43.715383,-79.405678
7,M5P,84700.0,19791.0,Central Toronto,Forest Hill North & West,43.696948,-79.411307


In [142]:
competitors = pd.read_csv('num_of_competitors.csv')
display(competitors)

,Neighbourhood,Venue
0,"University of Toronto, Harbord",20
1,"St. James Town, Cabbagetown",19
2,"Garden District, Ryerson",19
3,Church and Wellesley,18
4,Central Bay Street,18
...,...,...
60,"Lawrence Manor, Lawrence Heights",1
61,Hillcrest Village,1
62,"Guildwood, Morningside, West Hill",1
63,Downsview) East (CFB Toronto,1


In [143]:
final_combined_sorted = pd.merge(final_combined_sorted, competitors, on="Neighbourhood", how="left")

In [144]:
display(final_combined_sorted)

,Postalcode,Avg Income,Population,Borough,Neighbourhood,Latitude,Longitude,Venue
0,M4N,129000.0,16058.0,Central Toronto,Lawrence Park,43.728020,-79.388790,5.0
1,M4W,115500.0,15296.0,Downtown Toronto,Rosedale,43.679563,-79.377529,15.0
2,M4V,106200.0,19273.0,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,17.0
3,M4T,105100.0,10332.0,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,17.0
4,M8X,93900.0,10624.0,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0
...,...,...,...,...,...,...,...,...
91,M1G,33840.0,30894.0,Scarborough,Woburn,43.770992,-79.216917,1.0
92,M9V,33040.0,53878.0,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,NaN
93,M1V,32920.0,50825.0,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,NaN
94,M3N,31240.0,40846.0,North York,Downsview) Northwes,43.761631,-79.520999,NaN


In [145]:
final_combined_sorted.head(20)

,Postalcode,Avg Income,Population,Borough,Neighbourhood,Latitude,Longitude,Venue
0,M4N,129000.0,16058.0,Central Toronto,Lawrence Park,43.728020,-79.388790,5.0
1,M4W,115500.0,15296.0,Downtown Toronto,Rosedale,43.679563,-79.377529,15.0
2,M4V,106200.0,19273.0,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,17.0
3,M4T,105100.0,10332.0,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,17.0
4,M8X,93900.0,10624.0,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0
5,M4G,87500.0,19598.0,East York,Leaside,43.709060,-79.363452,3.0
6,M5R,85900.0,26197.0,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,16.0
7,M5M,85700.0,25495.0,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,4.0
8,M4R,85200.0,11909.0,Central Toronto,North Toronto West,43.715383,-79.405678,9.0
9,M5P,84700.0,19791.0,Central Toronto,Forest Hill North & West,43.696948,-79.411307,8.0


In [146]:

#  dataframe to store the results
high_education = pd.DataFrame(columns=["Postalcode", "Title", "High Education Count"])


# Loop through the postal codes
for i in postal_codes:
    url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E"
    params = {
        'SearchText': i,
        'DGUIDlist': '2021A0011'+i
    }

    # this part switches up the postal code parameter in the url
    url_parts = list(urlparse(url))
    query = dict(parse_qsl(url_parts[4]))
    query.update(params)
    
    url_parts[4] = urlencode(query)
    query_url = urlunparse(url_parts)
    
    try:
        # Open the URL and parse the HTML content
        html = urllib.request.urlopen(query_url)
        html_to_parse = BeautifulSoup(html, "html.parser")
        
        # Find all tables on the page
        tables = html_to_parse.find_all("table")
        
        # Check if any tables were found
        if len(tables) > 0:
            print(f"Number of tables found: {len(tables)}")
            title = f"2021A0011{i} - Bachelor's degree or higher - Counts - Total"
            
            # Find the relevant <td> with the title
            td = tables[0].find(attrs={"title": title})
            
            if td:
                high_edu = float(td.text.replace(",", ""))  # Convert population to a number
                print(f"Postalcode: {i}, High Education count: {high_edu}")
                high_education.loc[len(high_education.index)] = [i, title, high_edu]
            else:
                print(f"Title not found for postal code: {i}")
        else:
            print(f"No tables found for postal code: {i}")
    
    except Exception as e:
        print(f"Error processing postal code {i}: {e}")

# Display the dataframe
print(high_education)

Number of tables found: 1
Postalcode: M4N, High Education count: 8035.0
Number of tables found: 1
Postalcode: M4P, High Education count: 13115.0
Number of tables found: 1
Postalcode: M4R, High Education count: 6335.0
Number of tables found: 1
Postalcode: M4S, High Education count: 16940.0
Number of tables found: 1
Postalcode: M4T, High Education count: 5880.0
Number of tables found: 1
Postalcode: M4V, High Education count: 10760.0
Number of tables found: 1
Postalcode: M5N, High Education count: 7380.0
Number of tables found: 1
Postalcode: M5P, High Education count: 10595.0
Number of tables found: 1
Postalcode: M5R, High Education count: 14920.0
Number of tables found: 1
Postalcode: M4W, High Education count: 9360.0
Number of tables found: 1
Postalcode: M4X, High Education count: 7175.0
Number of tables found: 1
Postalcode: M4Y, High Education count: 20815.0
Number of tables found: 1
Postalcode: M5A, High Education count: 21690.0
Number of tables found: 1
Postalcode: M5B, High Education

In [147]:
display(high_education)

,Postalcode,Title,High Education Count
0,M4N,2021A0011M4N - Bachelor's degree or higher - C...,8035.0
1,M4P,2021A0011M4P - Bachelor's degree or higher - C...,13115.0
2,M4R,2021A0011M4R - Bachelor's degree or higher - C...,6335.0
3,M4S,2021A0011M4S - Bachelor's degree or higher - C...,16940.0
4,M4T,2021A0011M4T - Bachelor's degree or higher - C...,5880.0
...,...,...,...
91,M6C,2021A0011M6C - Bachelor's degree or higher - C...,10060.0
92,M6E,2021A0011M6E - Bachelor's degree or higher - C...,8710.0
93,M6M,2021A0011M6M - Bachelor's degree or higher - C...,5825.0
94,M6N,2021A0011M6N - Bachelor's degree or higher - C...,7190.0


In [148]:
high_education = high_education.drop(columns=["Title"], errors="ignore")

In [149]:

#  dataframe to store the results
fitness = pd.DataFrame(columns=["Postalcode", "Title", "Fitness workforce"])


# Loop through the postal codes
for i in postal_codes:
    url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E"
    params = {
        'SearchText': i,
        'DGUIDlist': '2021A0011'+i
    }

    # this part switches up the postal code parameter in the url
    url_parts = list(urlparse(url))
    query = dict(parse_qsl(url_parts[4]))
    query.update(params)
    
    url_parts[4] = urlencode(query)
    query_url = urlunparse(url_parts)
    
    try:
        # Open the URL and parse the HTML content
        html = urllib.request.urlopen(query_url)
        html_to_parse = BeautifulSoup(html, "html.parser")
        
        # Find all tables on the page
        tables = html_to_parse.find_all("table")
        
        # Check if any tables were found
        if len(tables) > 0:
            print(f"Number of tables found: {len(tables)}")
            title = f"2021A0011{i} - 31. Parks, recreation, leisure, fitness, and kinesiology - Counts - Total"
            
            # Find the relevant <td> with the title
            td = tables[0].find(attrs={"title": title})
            
            if td:
                fitnesswf = float(td.text.replace(",", ""))  # Convert population to a number
                print(f"Postalcode: {i}, Fitness Workforce count: {fitnesswf}")
                fitness.loc[len(fitness.index)] = [i, title, fitnesswf]
            else:
                print(f"Title not found for postal code: {i}")
        else:
            print(f"No tables found for postal code: {i}")
    
    except Exception as e:
        print(f"Error processing postal code {i}: {e}")

# Display the dataframe
display(fitness)

Number of tables found: 1
Postalcode: M4N, Fitness Workforce count: 100.0
Number of tables found: 1
Postalcode: M4P, Fitness Workforce count: 220.0
Number of tables found: 1
Postalcode: M4R, Fitness Workforce count: 60.0
Number of tables found: 1
Postalcode: M4S, Fitness Workforce count: 210.0
Number of tables found: 1
Postalcode: M4T, Fitness Workforce count: 65.0
Number of tables found: 1
Postalcode: M4V, Fitness Workforce count: 100.0
Number of tables found: 1
Postalcode: M5N, Fitness Workforce count: 90.0
Number of tables found: 1
Postalcode: M5P, Fitness Workforce count: 185.0
Number of tables found: 1
Postalcode: M5R, Fitness Workforce count: 165.0
Number of tables found: 1
Postalcode: M4W, Fitness Workforce count: 65.0
Number of tables found: 1
Postalcode: M4X, Fitness Workforce count: 75.0
Number of tables found: 1
Postalcode: M4Y, Fitness Workforce count: 180.0
Number of tables found: 1
Postalcode: M5A, Fitness Workforce count: 355.0
Number of tables found: 1
Postalcode: M5B, 

,Postalcode,Title,Fitness workforce
0,M4N,"2021A0011M4N - 31. Parks, recreation, leisure,...",100.0
1,M4P,"2021A0011M4P - 31. Parks, recreation, leisure,...",220.0
2,M4R,"2021A0011M4R - 31. Parks, recreation, leisure,...",60.0
3,M4S,"2021A0011M4S - 31. Parks, recreation, leisure,...",210.0
4,M4T,"2021A0011M4T - 31. Parks, recreation, leisure,...",65.0
...,...,...,...
91,M6C,"2021A0011M6C - 31. Parks, recreation, leisure,...",155.0
92,M6E,"2021A0011M6E - 31. Parks, recreation, leisure,...",165.0
93,M6M,"2021A0011M6M - 31. Parks, recreation, leisure,...",120.0
94,M6N,"2021A0011M6N - 31. Parks, recreation, leisure,...",130.0


In [150]:
display(fitness)

,Postalcode,Title,Fitness workforce
0,M4N,"2021A0011M4N - 31. Parks, recreation, leisure,...",100.0
1,M4P,"2021A0011M4P - 31. Parks, recreation, leisure,...",220.0
2,M4R,"2021A0011M4R - 31. Parks, recreation, leisure,...",60.0
3,M4S,"2021A0011M4S - 31. Parks, recreation, leisure,...",210.0
4,M4T,"2021A0011M4T - 31. Parks, recreation, leisure,...",65.0
...,...,...,...
91,M6C,"2021A0011M6C - 31. Parks, recreation, leisure,...",155.0
92,M6E,"2021A0011M6E - 31. Parks, recreation, leisure,...",165.0
93,M6M,"2021A0011M6M - 31. Parks, recreation, leisure,...",120.0
94,M6N,"2021A0011M6N - 31. Parks, recreation, leisure,...",130.0


In [151]:
fitness = fitness.drop(columns=["Title"], errors="ignore")

In [152]:
final_combined_sorted = pd.merge(final_combined_sorted, high_education, on="Postalcode", how="left")

In [153]:
final_combined_sorted = pd.merge(final_combined_sorted, fitness, on="Postalcode", how="left")

In [154]:
final_combined_sorted = final_combined_sorted.drop(columns=["Title"], errors="ignore")

In [156]:
final_combined_sorted.head(15)

,Postalcode,Avg Income,Population,Borough,Neighbourhood,Latitude,Longitude,Venue,High Education Count,Fitness workforce
0,M4N,129000.0,16058.0,Central Toronto,Lawrence Park,43.728020,-79.388790,5.0,8035.0,100.0
1,M4W,115500.0,15296.0,Downtown Toronto,Rosedale,43.679563,-79.377529,15.0,9360.0,65.0
2,M4V,106200.0,19273.0,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,17.0,10760.0,100.0
3,M4T,105100.0,10332.0,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,17.0,5880.0,65.0
4,M8X,93900.0,10624.0,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2.0,5475.0,80.0
5,M4G,87500.0,19598.0,East York,Leaside,43.709060,-79.363452,3.0,9715.0,150.0
6,M5R,85900.0,26197.0,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,16.0,14920.0,165.0
7,M5M,85700.0,25495.0,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,4.0,12470.0,205.0
8,M4R,85200.0,11909.0,Central Toronto,North Toronto West,43.715383,-79.405678,9.0,6335.0,60.0
9,M5P,84700.0,19791.0,Central Toronto,Forest Hill North & West,43.696948,-79.411307,8.0,10595.0,185.0


In [158]:
final_combined_sorted = final_combined_sorted.to_csv('torontofinaldf.csv', index=False)
